In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [32]:
from __future__ import print_function, division
import pandas
import src
import os, os.path
import matplotlib.pyplot as plt
import scipy.stats
from datetime import datetime
import re
from operator import itemgetter

In [3]:
def fake(*args, **kwargs):
    print('Fake called with', str(args), str(kwargs))
    sys.exit(1)

# fake out the create_model so we don't accidentally attempt to create data
src.common.create_model = fake

print(os.getcwd())
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
print(os.getcwd())

/home/cscorley/git/triage/notebooks
/home/cscorley/git/triage


In [8]:
logs=!find logs

In [13]:
logs=[log for log in logs if log.endswith('.log')]
flt=[log for log in logs if 'feature_location' in log]
dit=[log for log in logs if 'triage' in log]

flt, dit

(['logs/1-zookeeper-run-feature_location-file.log',
  'logs/1-mahout-run-feature_location-file.log',
  'logs/1-pig-run-feature_location-file.log',
  'logs/1-openjpa-run-feature_location-file.log',
  'logs/1-bookkeeper-run-feature_location-file.log',
  'logs/1-tika-run-feature_location-file.log'],
 ['logs/1-bookkeeper-run-triage-file.log',
  'logs/1-pig-run-triage-file.log',
  'logs/1-zookeeper-run-triage-file.log',
  'logs/1-openjpa-run-triage-file.log',
  'logs/1-tika-run-triage-file.log',
  'logs/1-mahout-run-triage-file.log'])

In [127]:
running = re.compile('(\d+-\d+-\d+ \d+:\d+:\d+).*common.* INFO Running (basic|temporal) evaluation( on the )?(.*)')
training = re.compile('(\d+-\d+-\d+ \d+:\d+:\d+).*gensim\.models\.ldamodel.* INFO running (batch|online) LDA training.*supplied corpus of (\d+) documents.*')
doc_topic = re.compile('(\d+-\d+-\d+ \d+:\d+:\d+).*common.* INFO Getting doc topic for corpus with length (\d+), by ids (.*)')
doc_topic_finish = re.compile('(\d+-\d+-\d+ \d+:\d+:\d+).*common.* INFO Returning doc topic of length (\d+)')
dtformat = '%Y-%m-%d %H:%M:%S'

log = 'logs/1-bookkeeper-run-feature_location-file.log'
for log in logs:
    with open(log) as f:
        dm_first = False
        rm_time = None

        times = []
        for line in f:
            rm = running.match(line)
            tm = training.match(line)
            dm = doc_topic.match(line)
            dmf = doc_topic_finish.match(line)

            if rm:
                if rm_time:
                    dm_first = False
                    rm_time = None
                    # new run, last one is finished

                rm_time = datetime.strptime(rm.group(1), dtformat)
                rm_type = rm.group(2)
                if rm_type == 'basic':
                    rm_type = rm.group(4)
                #print()
                #print()
                #rint('rm', rm_time, rm_type)

            if tm:
                tm_time = datetime.strptime(tm.group(1), dtformat)
                tm_type = tm.group(2)
                tm_size = int(tm.group(3))
                #print('tm', tm_time, tm_type, tm_size) 

                if tm_type == 'online':
                    dm_first = True

            if dm:
                dm_time = datetime.strptime(dm.group(1), dtformat)           
                dm_size = int(dm.group(2)) if dm.group(3) == 'None' else len(eval(dm.group(3)))
                dm_type = 'full' if dm.group(3) == 'None' else 'issue'
                if dm_type == 'full':
                    if dm_first:
                        dm_type = 'source'
                    else:
                        dm_first = True
                        dm_type = 'issues'

                #print('dm', dm_time, dm_size, dm_type)

                if tm_time:
                    times.append({'action': 'train',
                                  'experiment': rm_type,
                                  'action_type': tm_type,
                                  'documents': tm_size, 
                                  'total_seconds': (dm_time - tm_time).total_seconds()})
                                  #'raw_line': line})
                    tm_time = None


            if dmf:
                dmf_time = datetime.strptime(dmf.group(1), dtformat)
                dmf_size = int(dmf.group(2))

                #print('dmf', dmf_time, dmf_size)

                times.append({'action': 'infer',
                              'experiment': rm_type,
                              'action_type': dm_type, 
                              'documents': dm_size,
                              'total_seconds': (dmf_time - dm_time).total_seconds()})
                              #'raw_line': line})

        df = pandas.DataFrame(times)
        df.to_csv(log + '.csv',index_label="step")

In [128]:
df

,action,action_type,documents,experiment,total_seconds
0,train,batch,655,release-file,301.0
1,infer,issues,40,release-file,0.0
2,infer,source,655,release-file,5.0
3,train,batch,2469,changeset-True-True-False-True,2528.0
4,infer,issues,40,changeset-True-True-False-True,0.0
5,infer,source,655,changeset-True-True-False-True,7.0
6,train,online,1923,temporal,2277.0
7,infer,issue,1,temporal,0.0
8,infer,source,476,temporal,5.0
9,train,online,27,temporal,150.0


In [123]:
a.total_seconds()

7.0